In [52]:
import pandas as pd
import numpy as np

# Only read up to 4096 bytes, > 4096 has 100% malware rate
MAX_SIZE = 4096
# # MAX_SIZE = 98304
TOTAL_ROWS = 113636
USE_COLS = list(range(2, MAX_SIZE))
ROWS = 10000

train = pd.read_csv("./data/train/train.zip", nrows=ROWS, usecols=USE_COLS, header=None, names = list(range(0, MAX_SIZE)))
train = train.fillna(0, downcast='infer')

train_label = pd.read_csv("./data/train_label.csv", usecols=[1], nrows=ROWS)
train_label = np.reshape(train_label, (ROWS, ))

x_train = train.values
y_train = train_label.values
y_train = np.reshape(y_train, (ROWS, ))


In [53]:
import keras 
from keras import Sequential

# CHANGE THIS FILEPATH
filepath = 'mlp_model.h5'
mlp_model = keras.models.load_model(filepath)
ypred_mlp = mlp_model.predict(x_train)

ypred_mlp = np.reshape(ypred_mlp, (ROWS, ))


In [54]:
import lightgbm as lgb

# HERE USE THE LIGHTGBM WITH PARSER
bst_model = lgb.Booster(model_file='./model-99034.txt')
ypred_gbm = bst_model.predict(x_train, num_iteration=bst.best_iteration)


In [93]:
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import optimizers

batch_size = 64

class MyGenerator(Sequence):
    
    def __init__(self, x1_set, x2_set, y_set, batch_size):
        self.x1, self.x2, self.y = x1_set, x2_set, y_set,
        self.batch_size = batch_size

    def __len__(self):
        print(str(int(np.ceil(len(self.x1) / float(self.batch_size)))))
        return int(np.ceil(len(self.x1) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x1 = self.x1[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x2 = self.x2[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = np.zeros((len(batch_x1), 2))
        
        for i in range(len(batch_x1)):
            batch_x[i, :] = [batch_x1[i], batch_x2[i]]
        
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array(batch_x), np.array(batch_y)
    
    
seq = MyGenerator(ypred_gbm, ypred_mlp, y_train, batch_size)


model = Sequential()
model.add(Dense(1, input_dim=2, kernel_initializer=keras.initializers.Constant(value=1/2)))

adam=optimizers.Adam(lr=0.001)
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(y_train.shape)
model.fit_generator(generator=seq, epochs=50)


(10000,)
157
Epoch 1/50157

157/157 [==============================] - 3s 20ms/step - loss: 0.1216 - acc: 0.9633
Epoch 2/50
157/157 [==============================] - 0s 2ms/step - loss: 0.0932 - acc: 0.9870
Epoch 3/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0708 - acc: 0.9897
Epoch 4/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0529 - acc: 0.9914
Epoch 5/50
157/157 [==============================] - 0s 2ms/step - loss: 0.0385 - acc: 0.9922
Epoch 6/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0294 - acc: 0.9928
Epoch 7/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0262 - acc: 0.9928
Epoch 8/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0278 - acc: 0.9925
Epoch 9/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0290 - acc: 0.9925
Epoch 10/50
157/157 [==============================] - 0s 3ms/step - loss: 0.0280 - acc: 0.9927
Epoch 11/50
157/157 [==========

In [ ]:
TEST_TOTAL_ROWS = 133223
TEST_ROWS = TEST_TOTAL_ROWS
test = pd.read_csv("./data/actual_test.csv", nrows=TEST_ROWS, usecols=USE_COLS, header=None, names = list(range(0, MAX_SIZE)))
x_test = test.values
x_test = np.nan_to_num(x_test)

ytest_mlp = mlp_model.predict(x_test)
ytest_mlp = np.reshape(ytest_mlp, (TEST_ROWS, ))

ytest_gbm = bst_model.predict(x_test, num_iteration=bst.best_iteration)

for layer in model.layers:
    weights = layer.get_weights()
    
total = weights[0][0][0] + weights[0][1][0]
gbm_weight = weights[0][0][0] / total
mlp_weight = weights[0][1][0] / total


ytest_res = ytest_mlp*mlp_weight + ytest_gbm*gbm_weight